> Igor Sorochan DSU-31

# Loss function and Linear Regression optimization problem implementation without sklearn

In [14]:
from sklearn import datasets
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, accuracy_score

np.random.seed(0)

In [15]:
def linear_expression(x):
    return 4 * x + 7

objects_num = 5000
X = np.linspace(-5, 5, objects_num)
y = linear_expression(X) + np.random.randn(objects_num) * np.sqrt(10)
X = X[:, np.newaxis]
y = y[:, np.newaxis]

px.scatter(x=X.ravel(),y=y.ravel())

### The least squares method in a matrix form

Given: a linear regression model as  **matrix X [n, p]** - observations,   **column vector [n, 1]** y as dependent variable.
*  n observations 
*  p independent variables. 

The least squares method seeks to find the p x 1 column vector β that minimizes the sum of the squared residuals e, which is given by:

e = y - Xβ

The least squares estimate of β is the value that minimizes the sum of squared residuals e' * e, which is equivalent to minimizing the Euclidean norm of e. We can express this in matrix form as:
$$ \beta = (X^TX)^{-1}X^TY $$
where:
* $\beta$ is the least squares estimate of the parameters.
* $X^T$ is the transpose of the X matrix.  
* $(X^T X)^{-1}$ is the inverse of the product of $X^T$ and $X$.  
* $y$ is the column vector of actual values of the dependent variable.

|$$ \beta = (X^TX)^{-1}X^TY $$|
|:---|

Let's write it in code:

In [16]:
class Lin_regr():
    def __init__(self):
        self.coeffs= None
        print('Linear regression model is initialized')

    def fit(self, X, y):
        # number of observations
        self.rows = X.shape[0]
        
        XX = X

        # add eye column to the right, to simplify matrix operations with initial bias(intercept or B0)
        XX = np.hstack((X, np.ones((self.rows, 1))))

        # calculate coeffs as defined earlier
        self.coeffs = np.linalg.inv(XX.T @ XX) @ XX.T @ y
        print('Linear regression model is fitted')
        return 
        
    def predict(self, X):

        XX = np.hstack((X, np.ones((self.rows, 1))))

        y_pred = XX @ self.coeffs

        return y_pred
    
    def coefficients(self):
        print('Linear regression coefficients:')
        return self.coeffs

MSE should be approximately equal to 10.

In [17]:
lr = Lin_regr()
lr.fit(X,y)
mean_squared_error(lr.predict(X),y)

Linear regression model is initialized
Linear regression model is fitted


9.706220131730003

### Gradient descent optimization method

Inverse matrix $(X^TX)^{-1}$ used in calculating column vector $\beta$ doesn't always exist.  
Moreover, as the number of observations and features in our model increases,  
calculating the inverse matrix may becomes **costly** in terms of the use of computing power resources.  
That is why we often need to rely on **empirical methods** to find the minima of the loss function.  

One of very popular method is the **Stochastic Gradient Descent**.  
 It is a popular optimization algorithm used in machine learning to minimize the loss function of a model.  
 Instead of calculating the gradient of the entire dataset,  
 SGD updates the model parameters based on the gradient of a **random subset of the dataset**,  
 which is referred to as a **"mini-batch"**. 
 This makes the algorithm more efficient, especially when working with large datasets.  
 
 The **"stochastic"** part of the name comes from the fact that the **random selection** of the mini-batch  
 introduces some randomness into the optimization process,  
 which can help the algorithm escape local minimum and find a better overall solution.

### Gradient descent optimizations

The folowing code implements different optimization methods

In [131]:
class GradientOptimiser(Lin_regr):
    def __init__(self, **kwargs):
        '''
        GradientOptimiser inherits some parents methods and attributes (predict and coeffs)
        '''
        super().__init__(**kwargs) 
        self.coeffs = None
        
        print('GradientOptimiser is initialized')

    def fit(self, X, y, lr=0.01, max_iter=100, method = 'gradient', nbatches = 1, gamma= 0.9):
        '''
        The folowing code implements different optimization methods:
        - gradient descent
        - SGD with mimi - batches
        - RMSProp
        '''
        self.rows, n_features = X.shape
        self.nbatches = nbatches
        self.method = method
        self.gamma = gamma
        self.mselosses = []     # reset previously calculated losses
        self.gt = 0

        if self.coeffs is None:
            # increase number of features to handle the logit b0 (bias)
            # self.coeffs = np.zeros(n_features + 1)
            # random init with mean=0
            self.coeffs = np.random.random((n_features + 1)) - .5 

        # add a column of ones
        XX = np.hstack((X, np.ones((self.rows, 1)))) 
        

        # THE DESCENT
        for i in range(max_iter):
            W= self.coeffs
            y_pred = self.predict(X) # 

            self.mselosses.append(mean_squared_error(y_pred, y))   # fix mse loss

            if  self.method == 'nag':  
                shift = gamma * lr * self.gradient(XX, y, y_pred)
                self.coeffs -=  lr * self.gradient(XX - shift, y, y_pred)  

            elif self.method == 'nesterov':  # if gamma=0 nesteriv = sgd
                shift = gamma * lr * self.gradient(XX - (1-gamma) * W, y, y_pred)               
                self.coeffs -= lr * self.gradient(XX - shift, y, y_pred)  

                # shift = (1-gamma) * lr * self.gradient(XX*gamma - (1-gamma) * W, y, y_pred)               
                # self.coeffs -= lr * self.gradient(XX*gamma - shift, y, y_pred)  

            elif self.method == 'adagrad':
                shift = (1-gamma) * lr * self.gradient(XX, y, y_pred)
                self.gt += shift**2
                self.coeffs -= lr * self.gradient(XX - shift, y, y_pred) / np.sqrt(self.gt +1e-10)

            elif self.method == 'rmsprop':
                shift = (1-gamma) * lr * self.gradient(XX - gamma * W, y, y_pred)
                self.gt += shift**2
                self.coeffs -= lr * self.gradient(XX - shift, y, y_pred) / np.sqrt(self.gt +1e-10)
             
            else:
                #  learning rate  * gradient
                self.coeffs -= lr * self.gradient(XX, y, y_pred)

        return 


    def gradient(self, XX, y, y_pred ):

        if self.method == 'gradient':  # gradient descent over full data set
               
            gradient =  2 * XX * (y_pred[:, np.newaxis] - y) 
        else:
            # generate batches by random indexes 
            np.random.seed(42)
            i = np.random.choice(np.arange(self.rows), size= int(self.rows / self.nbatches), replace= False)          
            gradient =  2 * XX[i] * (y_pred[i][:, np.newaxis] - y[i])

        gradient = gradient.mean(axis=0) # averaging
        return gradient

    def get_mseloss(self):
        return self.mselosses    

Define a plot function.

In [19]:
def plot_losses(losses, title, legend):
    myline = px.line(losses, title= title, height= 700, labels= 'Gradient descent')
    fig = go.Figure(myline)

    fig.update_layout(
        title= title,
        xaxis_title='Number of iterations',
        yaxis_title="Loss function",
        legend_title= legend,
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )  
    fig.show()  

### Gradient Descent over full dataset

In [97]:
optimiser = GradientOptimiser()

losses= {}
for lr in np.logspace(-1, -4, 4):
  optimiser.coeffs = None
  optimiser.fit(X, y, lr = lr , max_iter=300, method = 'gradient')  

  losses.setdefault (lr, optimiser.get_mseloss())

for k,v in losses.items():
    print (f'Learning rate: {k:f}, MSE:{round(v[-1],6)}')
plot_losses(losses, title= 'Gradient Descent over full dataset. Loss function', legend = 'learning rate:')

Linear regression model is initialized
GradientOptimiser is initialized
Learning rate: 0.100000, MSE:9.70622
Learning rate: 0.010000, MSE:9.706475
Learning rate: 0.001000, MSE:26.09973
Learning rate: 0.000100, MSE:108.739419


### Gradient descent with random batches (SGD)

In [101]:
optimiser = GradientOptimiser()

losses= {}
# for nbatches in [1, 10, 30, 50]:
for lr in np.logspace(-1, -6, 6):
  optimiser.coeffs = None
  optimiser.fit(X, y, lr = lr , max_iter=1000, method = 'sgd', nbatches= 10)

  losses.setdefault (lr, optimiser.get_mseloss())

for k,v in losses.items():
    # print (f'Number of batches: {k}, MSE:{round(min(v),3)}')
    print (f'learning rate: {k}, MSE:{round(v[-1],6)}')
plot_losses(losses, title= 'Stochastic Gradient Descent. Loss function', legend = 'learning rate:')

Linear regression model is initialized
GradientOptimiser is initialized
learning rate: 0.1, MSE:9.755785
learning rate: 0.01, MSE:9.755785
learning rate: 0.001, MSE:11.172812
learning rate: 0.0001, MSE:56.194852
learning rate: 1e-05, MSE:175.881274
learning rate: 1e-06, MSE:214.828319


With number of **batches = 1**, SGD is actually a **Gradient Descent** on full dataset.  
As the number of batches increased so the stochastic behavoir of SGD becomes more obvious.  
In extreme case (batches = observations) the convergency level is higher.  The loss function may have found a local minima.

Let's play with parameters. I set very low learning rate and significantly increase the number of iterations.

### Nesterov Accelerated Gradient


<p style="align: center;"><img align=center src="https://tex.s2cms.ru/svg/%0Av_t%20%3D%20%5Cgamma%20v_%7Bt-1%7D%20%2B%20%5Ceta%20%5Cnabla_%5Ctheta%20J(%20%5Ctheta)%0A" height=50/></p>

<p style="align: center;"><img align=center src="https://tex.s2cms.ru/svg/%0A%5Ctheta%20%3D%20%5Ctheta%20-%20v_t%0A" height=50/></p>


In [135]:
optimiser = GradientOptimiser()
losses= {}
for gamma in [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1]:
# for lr in np.logspace(-1, -6, 6):
  # reset already learned coefficients 
  optimiser.coeffs = None
  optimiser.fit(X, y, lr = 3e-2, max_iter=50, method = 'nag', nbatches= 10, gamma = gamma)
  # optimiser.fit(X, y, lr = lr, max_iter=30, method = 'nag', nbatches= 10, gamma = 0.9)
  losses.setdefault (gamma, optimiser.get_mseloss())

for k,v in losses.items():
    print (f'Gamma: {k}, MSE:{round(v[-1],6)}')
plot_losses(losses, title= 'Nesterov Accelerated Gradient. Loss function', legend = 'gamma:')

Linear regression model is initialized
GradientOptimiser is initialized
Gamma: 0, MSE:10.02609
Gamma: 0.1, MSE:10.009446
Gamma: 0.2, MSE:9.99449
Gamma: 0.3, MSE:9.980994
Gamma: 0.4, MSE:9.968768
Gamma: 0.5, MSE:9.957651
Gamma: 0.7, MSE:9.938225
Gamma: 0.8, MSE:9.929704
Gamma: 0.9, MSE:9.921859
Gamma: 1, MSE:9.914618


### Nesterov momentum

<p style="align: center;"><img align=center src="https://tex.s2cms.ru/svg/%0Av_t%20%3D%20%5Cgamma%20v_%7Bt-1%7D%20%2B%20%5Ceta%20%5Cnabla_%5Ctheta%20J(%20%5Ctheta%20-%20%5Cgamma%20v_%7Bt-1%7D%20)%0A" height=50/></p>

<p style="align: center;"><img align=center src="https://tex.s2cms.ru/svg/%0A%5Ctheta%20%3D%20%5Ctheta%20-%20v_t%0A" height=50/></p>


In [134]:
optimiser = GradientOptimiser()
losses= {}
for gamma in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9, 1]:
  # reset already learned coefficients 
  optimiser.coeffs = None
  optimiser.fit(X, y, lr = 3e-2, max_iter=50, method = 'nesterov', nbatches= 10, gamma = gamma)
  losses.setdefault (gamma, optimiser.get_mseloss())
for k,v in losses.items():
  print (f'Gamma: {k}, MSE:{round(v[-1],6)}')
plot_losses(losses, title= 'Nesterov momentum. Loss function', legend = 'gamma:')

Linear regression model is initialized
GradientOptimiser is initialized
Gamma: 0, MSE:10.02609
Gamma: 0.1, MSE:10.058964
Gamma: 0.2, MSE:10.083857
Gamma: 0.3, MSE:10.09689
Gamma: 0.4, MSE:10.095838
Gamma: 0.5, MSE:10.080868
Gamma: 0.7, MSE:10.0207
Gamma: 0.8, MSE:9.983871
Gamma: 0.9, MSE:9.947681
Gamma: 1, MSE:9.914618


### Adagrad (Adaptive gradient)

In [136]:
optimiser = GradientOptimiser()
losses= {}
# for gamma in [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9]:
for lr in np.logspace(-1, -6, 6):

  optimiser.coeffs = None
  optimiser.fit(X, y, lr = lr, max_iter=30, method = 'adagrad', nbatches= 10, gamma = .9)
  losses.setdefault (lr, optimiser.get_mseloss())

for k,v in losses.items():
    print (f'learning rate: {k}, MSE:{round(v[-1],5)}')

plot_losses(losses, title= 'Adaptive gradient. Loss function', legend= 'learning rate:')

Linear regression model is initialized
GradientOptimiser is initialized
learning rate: 0.1, MSE:9.75579
learning rate: 0.01, MSE:9.75579
learning rate: 0.001, MSE:9.75579
learning rate: 0.0001, MSE:9.75579
learning rate: 1e-05, MSE:9.75579
learning rate: 1e-06, MSE:9.76145


### RMSProp (Route Mean Square Propogation)

In [115]:
optimiser = GradientOptimiser()
losses= {}
# for gamma in [0.1, 0.2, 0.3, 0.4, 0.5, 0.7,0.8, .9]:
for lr in np.logspace(-1, -6, 6):

  optimiser.coeffs = None
  optimiser.fit(X, y, lr = lr, max_iter=30, method = 'rmsprop', nbatches= 10, gamma = 0.9)
  losses.setdefault (lr, optimiser.get_mseloss())

for k,v in losses.items():
    print (f'learning rate: {k}, MSE:{round(v[-1],5)}')

plot_losses(losses, title= 'Route Mean Square Propogation. Loss function', legend = 'learning rate:')

Linear regression model is initialized
GradientOptimiser is initialized
learning rate: 0.1, MSE:9.75579
learning rate: 0.01, MSE:9.75579
learning rate: 0.001, MSE:9.75579
learning rate: 0.0001, MSE:9.75579
learning rate: 1e-05, MSE:9.75579
learning rate: 1e-06, MSE:9.76312


$\gamma$ 

For small $\gamma$ and relatively large $\eta$ , the momentum is so large that the loss function is unlikely to converge